In [127]:
import pandas_datareader as pdf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
import seaborn as sns
from scipy import stats
plt.rc("figure", figsize=(16, 8))
plt.rc("font", size=14)

In [128]:
fed_data1 = pdf.get_data_fred(['FEDFUNDS','UNRATE', 'INDPRO', 'W823RC1', 'CPILFESL', 'FRGSHPUSM649NCIS'])
fed_data1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 59 entries, 2017-05-01 to 2022-03-01
Freq: MS
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   FEDFUNDS          59 non-null     float64
 1   UNRATE            59 non-null     float64
 2   INDPRO            59 non-null     float64
 3   W823RC1           58 non-null     float64
 4   CPILFESL          59 non-null     float64
 5   FRGSHPUSM649NCIS  59 non-null     float64
dtypes: float64(6)
memory usage: 3.2 KB


In [129]:
fed_data2 = pdf.get_data_fred(['DCOILBRENTEU', 'WM2NS', 'FRGSHPUSM649NCIS'])
fed_data2 = fed_data2.resample('D').asfreq().fillna(method='ffill').reset_index()
fed_data2.describe()

,DCOILBRENTEU,WM2NS,FRGSHPUSM649NCIS
count,1818.000000,1814.000000,1814.000000
mean,63.281249,16649.273208,1.158337
std,16.886389,2900.765740,0.074395
min,9.120000,13450.300000,0.923000
25%,53.167500,14152.300000,1.128000
50%,64.280000,15135.700000,1.171000
75%,72.357500,19423.000000,1.203000
max,133.180000,21823.400000,1.307000


In [130]:
fed_data2.dropna().tail()

,DATE,DCOILBRENTEU,WM2NS,FRGSHPUSM649NCIS
1813,2022-04-14,110.83,21768.8,1.203
1814,2022-04-15,110.83,21768.8,1.203
1815,2022-04-16,110.83,21768.8,1.203
1816,2022-04-17,110.83,21768.8,1.203
1817,2022-04-18,110.83,21768.8,1.203


In [131]:
covid_data = pd.read_csv('covid.csv', sep = ',')
covid_df = pd.DataFrame(covid_data).set_index('DATE')
covid_df.index = pd.to_datetime(covid_df.index)
covid_df.index.name = 'DATE'

In [132]:
merge_df = [fed_data1, fed_data2, covid_df]
new_df = covid_df.merge(fed_data1, on='DATE').dropna().merge(fed_data2, on='DATE')
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                25 non-null     datetime64[ns]
 1   Confirmed           25 non-null     int64         
 2   FEDFUNDS            25 non-null     float64       
 3   UNRATE              25 non-null     float64       
 4   INDPRO              25 non-null     float64       
 5   W823RC1             25 non-null     float64       
 6   CPILFESL            25 non-null     float64       
 7   FRGSHPUSM649NCIS_x  25 non-null     float64       
 8   DCOILBRENTEU        25 non-null     float64       
 9   WM2NS               25 non-null     float64       
 10  FRGSHPUSM649NCIS_y  25 non-null     float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 2.3 KB


In [133]:
reg4 = 'CPILFESL~Confirmed+UNRATE+INDPRO+FEDFUNDS+W823RC1+FRGSHPUSM649NCIS+DCOILBRENTEU+WM2NS+FRGSHPUSM649NCIS'
reg4output=smf.ols(reg4, new_df).fit()
print(reg4output.summary())

PatsyError: Error evaluating factor: NameError: name 'FRGSHPUSM649NCIS' is not defined
    CPILFESL~Confirmed+UNRATE+INDPRO+FEDFUNDS+W823RC1+FRGSHPUSM649NCIS+DCOILBRENTEU+WM2NS+FRGSHPUSM649NCIS
                                                      ^^^^^^^^^^^^^^^^

In [ ]:
new_df.describe()

In [ ]:
pred_ols = reg4output.get_prediction()


iv_l = pred_ols.summary_frame()["obs_ci_lower"]
iv_u = pred_ols.summary_frame()["obs_ci_upper"]

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(x, y, "o", label="data")
ax.plot(x, reg4output.fittedvalues, "r--.", label="OLS")
ax.plot(x, iv_u, "r--")
ax.plot(x, iv_l, "r--")
ax.legend(loc="best")

In [ ]:
Y = new_df['CPILFESL']
X = new_df[['W823RC1',
 'UNRATE',
 'INDPRO',
 'FEDFUNDS',
 'DCOILBRENTEU',
 'FRGSHPUSM649NCIS',
 'Confirmed',
 'WM2NS',
]]
X = sm.add_constant(X)

In [ ]:
ks = sm.OLS(Y, X)
ks_res =ks.fit()
ks_res.summary()

In [ ]:
sns.pairplot(ks);

In [ ]:
sns.distplot(reg4output.resid, fit=stats.norm);

In [ ]:
sm.qqplot(reg4output.resid, line='s');